<a href="https://colab.research.google.com/github/Ironblade1/API_assignment/blob/main/Extracting%20Data%20in%20form%20of%20JSON/Text%20from%20Multiple%20PDFs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing libraries

In [4]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.9 MB/s eta 0:00:00


In [5]:
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 3.6 MB/s eta 0:00:00


In [6]:
from google.colab import userdata    # getting API key
key = userdata.get('openAi_API_key')

In [7]:
import base64
import requests

# OpenAI API Key
api_key = key

# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# Function to extract the data from each Image
def extract_data_from_eachImage(image_path):

  # image_path = "/content/Capture.PNG"

  # Getting the base64 string
  base64_image = encode_image(image_path)

  headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
  }

  # change the payload as required
  payload = {
    "model": "gpt-4o",
    "messages": [
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": "Extract the entire data from the image in json format , do not miss any of line or table, extract all data from top to bottom of the image "
          },
          {
            "type": "image_url",
            "image_url": {
              "url": f"data:image/jpeg;base64,{base64_image}",
              "detail": "high"
            }
          }
        ]
      }
    ]
  }

  response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

  # print(response.json())
  # print(response.json()['choices'][0]['message']['content'])
  return response.json()['choices'][0]['message']['content']


In [8]:
!apt-get install -y poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.5 [186 kB]
Fetched 186 kB in 1s (312 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 123588 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.5_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.5) ...
Setting up poppler-utils (22.02.0-2ubuntu0.5) ...
Processing triggers for man-db (2.10.2-1) ...


In [9]:
!pip install pdf2image

In [ ]:
!pip install PyPDF2

In [13]:
from pdf2image import convert_from_path
import os

# function for extracting the data from each PDF
def extract_data_from_eachPDF(pdf):
  # pdf_path = "/content/19590102.pdf"
  input_pdf_dir = "/content/drive/MyDrive/data_for_extracting/input_pdfs/"   #directory where all input pdfs are there
  pdf_path = input_pdf_dir + pdf          # path of the pdf

  image_dir = "/content/drive/MyDrive/data_for_extracting/image"        #Image directory where all image folders are generated for each pdf

  image_dir_path = os.path.join(image_dir, f"{pdf[:-4]}")       #Image directory path of each pdf
  # os.makedirs(image_dir, exist_ok=True)

  if not os.path.exists(image_dir_path):
    os.mkdir(image_dir_path)             # creating a directory if its already isn't exist

  # Convert PDF to images
  images = convert_from_path(pdf_path)

  data = ""  # storing all the data

  # Iterate over each image (page of the PDF)
  for i, image in enumerate(images):
      # Save the image
      # image_path = os.path.join(image_dir, f"/page_{i+1}.png")
      image_path = image_dir_path + f"/page_{i+1}.png"
      image.save(image_path, "PNG")
      print(f"Saved image: {image_path}")

      # print(f"Image {i+1} size: {image.size}")
      # image_path = image_dir + f"/page_{i+1}.png"

      res = extract_data_from_eachImage(image_path)

      # res = Validate_Json(res) ### use this fucntion if required
      data += res

  # creating a json file for this pdf and saving the data
  extract_data_dir = "/content/drive/MyDrive/data_for_extracting/output_dir"
  extract_file_path = os.path.join(extract_data_dir, f"/{pdf[:-4]}.json")
  # output.save(extract_file_path, "JSON")

  output_file_path = extract_data_dir + f"/{pdf[:-4]}.json"
  with open(output_file_path, 'w') as file:
    file.write(data)

  print(f"Saved JSON File: {output_file_path}")
  # print(data)

In [ ]:
#function for validating the json
def Validate_Json(data):

  headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
  }

  payload = {
    "model": "gpt-4o-mini",
    "messages": [
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": "convert this JSON data into validate JSON . its having bracket parenthesis error solve every error and return valid JSON data. Note: do not add any text from your side and no need of adding comment"
          },
          {
            "type": "text",
            "text": data
          }
        ]
      }
    ]
  }

  response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

  # print(response.json())
  # print(response.json()['choices'][0]['message']['content'])
  json_data = response.json()['choices'][0]['message']['content']
  st = 0
  for i in range(len(json_data)):
      if json[i] == '{':
          st = i
          break

  end = 0
  for i in range(len(json_data) - 1, -1, -1):
      if json[i] == '}':
          end = i + 1
          break

  corrected_json = json_data[st:end]

  return corrected_json


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
def main():
  pdfs = os.listdir("/content/drive/MyDrive/data_for_extracting/input_pdfs")
  for i,pdf in enumerate(pdfs):
    extract_data_from_eachPDF(pdf)
    # print(pdf)

main()

Saved image: /content/drive/MyDrive/data_for_extracting/image/19910102/page_1.png
Saved image: /content/drive/MyDrive/data_for_extracting/image/19910102/page_2.png
Saved image: /content/drive/MyDrive/data_for_extracting/image/19910102/page_3.png
Saved image: /content/drive/MyDrive/data_for_extracting/image/19910102/page_4.png
Saved image: /content/drive/MyDrive/data_for_extracting/image/19910102/page_5.png
Saved image: /content/drive/MyDrive/data_for_extracting/image/19910102/page_6.png
Saved image: /content/drive/MyDrive/data_for_extracting/image/19910102/page_7.png
Saved image: /content/drive/MyDrive/data_for_extracting/image/19910102/page_8.png
Saved image: /content/drive/MyDrive/data_for_extracting/image/19910102/page_9.png
Saved image: /content/drive/MyDrive/data_for_extracting/image/19910102/page_10.png
Saved image: /content/drive/MyDrive/data_for_extracting/image/19910102/page_11.png
Saved image: /content/drive/MyDrive/data_for_extracting/image/19910102/page_12.png
Saved image: 